In [214]:
# %load ../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)

%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

#import seaborn as sns
#sns.set_context('notebook')
#sns.set_style('darkgrid')

### Reshaping data: Melt & Pivot

In [215]:
df = pd.DataFrame({'Machine1':[53,47,46,50,49],
                   'Machine2':[61,55,52,58,54],
                   'Machine3':[51,51,49,54,50],
                   'Operator':['A','B','C','D','E']})

print(df)

   Machine1  Machine2  Machine3 Operator
0        53        61        51        A
1        47        55        51        B
2        46        52        49        C
3        50        58        54        D
4        49        54        50        E


#### Melt: from wide to long

In [216]:
molten_df = pd.melt(df, id_vars='Operator', value_vars=['Machine1', 'Machine2', 'Machine3'])
print(molten_df)

   Operator  variable  value
0         A  Machine1     53
1         B  Machine1     47
2         C  Machine1     46
3         D  Machine1     50
4         E  Machine1     49
5         A  Machine2     61
6         B  Machine2     55
7         C  Machine2     52
8         D  Machine2     58
9         E  Machine2     54
10        A  Machine3     51
11        B  Machine3     51
12        C  Machine3     49
13        D  Machine3     54
14        E  Machine3     50


####  Pivot: from long to wide

In [217]:
pivot_df = molten_df.pivot(index='Operator', columns='variable', values='value') # reverse above 'melt' operation
print(pivot_df) # Operator is set as index, but this can be undone by 'reset_index'

variable  Machine1  Machine2  Machine3
Operator                              
A               53        61        51
B               47        55        51
C               46        52        49
D               50        58        54
E               49        54        50


### Reshaping data: Stack & Unstack

In [218]:
# Creating DataFrame with MultiIndexes on both axis
idx1 = pd.MultiIndex.from_product([['A','B','C'],['XX', 'YY']], names=['Operator', 'Facility'])

arrays = np.array(['OB11', 'OB11', 'HH90']), np.array(['M1', 'M2', 'M3'])
idx2 = pd.MultiIndex.from_arrays(arrays, names=['Machine type', 'MachineID'])

df2 = pd.DataFrame([[53,47,46],
                   [50,49,48],
                   [61,55,52],
                   [58,54,57],
                   [51,51,49],
                   [54,50,55]], index=idx1, columns=idx2)
df2

Machine type      OB11     HH90
MachineID           M1  M2   M3
Operator Facility              
A        XX         53  47   46
         YY         50  49   48
B        XX         61  55   52
         YY         58  54   57
C        XX         51  51   49
         YY         54  50   55

#### Unstack : move an index to the columns

In [219]:
df2.unstack() #by default pandas unstacks last level (Facility in this case)

Machine type OB11             HH90    
MachineID      M1      M2       M3    
Facility       XX  YY  XX  YY   XX  YY
Operator                              
A              53  50  47  49   46  48
B              61  58  55  54   52  57
C              51  54  51  50   49  55

In [229]:
df2.unstack('Operator')

Machine type OB11                     HH90        
MachineID      M1          M2           M3        
Operator        A   B   C   A   B   C    A   B   C
Facility                                          
XX             53  61  51  47  55  51   46  52  49
YY             50  58  54  49  54  50   48  57  55

In [238]:
# unstack all levels passing a list of level numbers. Identical to: df2.unstack(['Facility', 'Operator'])
df2.unstack([1,0])

Machine type  MachineID  Facility  Operator
OB11          M1         XX        A           53
                                   B           61
                                   C           51
                         YY        A           50
                                   B           58
                                   C           54
              M2         XX        A           47
                                   B           55
                                   C           51
                         YY        A           49
                                   B           54
                                   C           50
HH90          M3         XX        A           46
                                   B           52
                                   C           49
                         YY        A           48
                                   B           57
                                   C           55
dtype: int64

#### Stack: move a column into the index

In [221]:
df2.stack() #by default pandas unstacks last level (MachineID in this case)

Machine type                 HH90  OB11
Operator Facility MachineID            
A        XX       M1          NaN    53
                  M2          NaN    47
                  M3           46   NaN
         YY       M1          NaN    50
                  M2          NaN    49
                  M3           48   NaN
B        XX       M1          NaN    61
                  M2          NaN    55
                  M3           52   NaN
         YY       M1          NaN    58
                  M2          NaN    54
                  M3           57   NaN
C        XX       M1          NaN    51
                  M2          NaN    51
                  M3           49   NaN
         YY       M1          NaN    54
                  M2          NaN    50
                  M3           55   NaN

In [222]:
df2.stack(0)

MachineID                       M1  M2  M3
Operator Facility Machine type            
A        XX       HH90         NaN NaN  46
                  OB11          53  47 NaN
         YY       HH90         NaN NaN  48
                  OB11          50  49 NaN
B        XX       HH90         NaN NaN  52
                  OB11          61  55 NaN
         YY       HH90         NaN NaN  57
                  OB11          58  54 NaN
C        XX       HH90         NaN NaN  49
                  OB11          51  51 NaN
         YY       HH90         NaN NaN  55
                  OB11          54  50 NaN

In [223]:
df2.stack([0,1]) # unstacking all levels (Machine type and MachineID)

Operator  Facility  Machine type  MachineID
A         XX        HH90          M3           46
                    OB11          M1           53
                                  M2           47
          YY        HH90          M3           48
                    OB11          M1           50
                                  M2           49
B         XX        HH90          M3           52
                    OB11          M1           61
                                  M2           55
          YY        HH90          M3           57
                    OB11          M1           58
                                  M2           54
C         XX        HH90          M3           49
                    OB11          M1           51
                                  M2           51
          YY        HH90          M3           55
                    OB11          M1           54
                                  M2           50
dtype: float64

### Swapping index hierarchies (rows & columns)

In [224]:
df2

Machine type      OB11     HH90
MachineID           M1  M2   M3
Operator Facility              
A        XX         53  47   46
         YY         50  49   48
B        XX         61  55   52
         YY         58  54   57
C        XX         51  51   49
         YY         54  50   55

In [225]:
# Swap the indexes on the rows using the index position/number. 
df2.swaplevel(0,1)

Machine type      OB11     HH90
MachineID           M1  M2   M3
Facility Operator              
XX       A          53  47   46
YY       A          50  49   48
XX       B          61  55   52
YY       B          58  54   57
XX       C          51  51   49
YY       C          54  50   55

In [226]:
# Swap the indexes on the columns using the labels. By default, the function swaps levels on row index. That's
# why you need to explicitly indicate the column axis in this case.
df2.swaplevel('Machine type', 'MachineID', axis=1)

MachineID           M1   M2   M3
Machine type      OB11 OB11 HH90
Operator Facility               
A        XX         53   47   46
         YY         50   49   48
B        XX         61   55   52
         YY         58   54   57
C        XX         51   51   49
         YY         54   50   55